# Prediction of Action Unit

### Intention:
This notebook contains the analysis of fmri data aquired in Marburg.
The analysis is set up to conceptionally replicate the findings of the study "A Neural Basis of Facial Action Recognition in Humans" by Srinivasan et al. in 2016. (see link for more information)

### Underlying Experiment:
Our paradigma is implemented in psychopy3 and gets presented to the participants while they are in the scanner.
In the experiment participants see short videos (1.5 s) of human avatars. These avatars change their facial expression througout the video from neutral into several kinds of emotional facial expressions. The emotions used for this paradigm are:  

1. happy  
2. disgusted  
3. happily surprised  
4. happily disgusted  
5. angriliy surprised  
6. fearfully surprised  
7. sadly fearful  
8. fearfully disgusted  

In one block Participants see 6 videos of different avatars moving into the same emotion. Participants then see the names of 2 emotions and have to select the one that rather decribes the videos presneted beforehand. The experiment conists of 9 runs with 16 blocks each.

### Analysis:
As done by Srinivasan et al. **we used machine learning to predict the presence/absence of AU's** (for more detail on AU's and their link to emotions check:...) from our neuroimaging data. To do so we first have to train a machine in a supervised manner. **Supervised learning algorithms require a set of data with labels on which they can train/learn.**  
To test if they learned correctly one can use unlabeled data from the same distribution and check the accuracy. This splitting of the data is called **cross-validation** and there are several ways in which one can split the data.
In another notebook (see..link again) I extracted the time information from the log-files in order to beeing able to use supervised machine learning tools.  
The whole data analysis was done using python 3.6. 
Data structure is in compliance with BIDS stardarts, **This structure  is required for the functions defined in the following analysis**.
**Specifically in this pipeline the following steps were made for every subject:**  

1. Several anatomical mask in MNI152NLin2009cAsym-space were created using the **Harvard-Oxford atlas**
2. The 4D-preprocessed NIFTI-data ist transformed into a 2D-array, where the first dimension represents the timepoints and the second dimesion represents activation of every voxel within a given mask. For this transformation the **NiftiMasker** function from the nilearn package was applied to the data given the before created mask of the pSTS
3. For computaional reasosn we then reduced the dimensionality of this 2D-array such that still 95% of the variance can be explained. The tool used for this is a **principle component analysis.**(optional)
4. We then trained a **linear Support Vector Machine** (other classifiers possible) on 8 runs of the experimental data and tested it on the remaining run to get accuracy rates. As there are 9 ways of splitting the data (leave every run out once) we get 9 different scores. Computing the average across these 9 accuracies leaves us with a mean accuracy. This was done for the four AU that we examine in our study.  


In [1]:
%matplotlib inline

## Import Modules 

## we have to import all packages that we will use in the following analysis


### path should be the absolute path (less errorprone than relative path for some packages) of the directory containing your data in the BIDS format
path = "/media/lmn/86A406A0A406933B/Aaron_MA/data_bids/"
### List to iterate over all subjects 
subject_ids = ["02","03","04"]
#### List used to iterate over AU in the analysis
AUs = ["AU1","AU2","AU12","AU20"]

In [2]:
## we have to import all packages that we will use in the following analysis
import nilearn.image
from nilearn import plotting
from nilearn.image import mean_img
from nilearn.input_data import NiftiMasker
import pandas as pd
import numpy as np
import nibabel as nib
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneGroupOut, StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_validate, permutation_test_score
from sklearn.decomposition import PCA
from nilearn import datasets
from nilearn import plotting, image
from nilearn.image import load_img, math_img
from nilearn.masking import intersect_masks
import fpdf
import matplotlib.pyplot as plt

#path should be the absolute path (less errorprone than relative path for some packages) 
#of the directory containing your data in the BIDS format
path = "/media/lmn/86A406A0A406933B/Aaron_MA/data_bids/"
#List to iterate over all subjects 
subject_ids = ["02","03","04","06","07","08","09","10","11"]
#List used to iterate over AU in the analysis
AUs = ["AU1","AU2","AU12","AU20"]
#dataframe to store the results
results_df = pd.DataFrame(columns=['sid','AU','score','pvalue','permutation_score'])

## Initialise the data frame for the storage of the results. This dataframe is saved to your BIDS directory as csv-file later in the analysis workflow (after it is filled with the respective results)

In [3]:
#fill the dataframe with subject Ids and AU rows, which then will be filled with the accuracies afterwards
i=0
for index in range(0, (4*len(subject_ids))):
    
    if index%4 == 0 and index != 0:
        i= i+1
   
    results_df.at[index,'sid'] = subject_ids[i]
    results_df.at[index,'AU'] = AUs[index%4]


## Define the data_tranformer function
This function will perform the transformation of 4D Nifti-files to 2D numpy arrays. It requires a subject_id or a list of subject_ids. If sub is a string (a single subject_id) this function will mask the 4D data given the mask one chooses in the parameters. Setting the parameter method you can choose to average the signal over each block or take into account every single scan. If you want to add a PCA you can choose between 3 options:

1.  PCA before the cutting of uninteresting trials
2.  PCA after the  cutting  
3.  If you choose to average over blocks you can do the PCA after this averaging

If sub is a list of subject_ids this function will perform data_transformation for every subject in this list and will concatenate the arrays along the time axis. A PCA cannot be chosen as we might get into dimensionality troubles (every subject might have sligthly different number of PC's). 

In [5]:
def data_transformer(sub, time_shift = 0,method='single_scan',mask='pSTS',pca=True,pca_position='after_cutting'):
    mask_path = path+'/derivatives/masks/'
    
    pSTS_real = mask_path +"pSTS_real.nii"
    fusiform_pSTS_mask= mask_path+'ff_pSTS.nii'
    whb_mask = mask_path+'whb.nii'
    lingual = mask_path+'lingual_mask.nii'
    pSTS_lingual = mask_path+'pSTS_lingual.nii'
    fusiform = mask_path+'fusiform.nii'
    pSTS_small = mask_path+'pSTS_small.nii'
    
    if isinstance(sub, str) == True:
        sid = sub
        data =  nilearn.image.load_img(path+"derivatives/fmriprep/sub-"+sid+"/func/sub-"+sid+"_task-video_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz", dtype="auto")
        #get the whole brain mask from every subjects folder
        mask_filename =  path+"derivatives/fmriprep/sub-"+sid+"/func/sub-"+sid+"_task-video_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"

        if mask == 'pSTS':
            #we use a masker-object to transform the data within the mask into a 2D-array
            #we need to create a masker-object using the beforehand created mask of the pSTS as mask_img 
            masker = NiftiMasker(mask_img=pSTS_real, standardize=True)
        if mask == 'pSTS_lingual':
            #we use a masker-object to transform the data within the mask into a 2D-array
            #we need to create a masker-object using the beforehand created mask of the pSTS as mask_img 
            masker = NiftiMasker(mask_img=pSTS_lingual, standardize=True)
        if mask == 'fusiform':
            #we use a masker-object to transform the data within the mask into a 2D-array
            #we need to create a masker-object using the beforehand created mask of the pSTS as mask_img 
            masker = NiftiMasker(mask_img=fusiform, standardize=True)
        if mask == 'pSTS_small':
            #we use a masker-object to transform the data within the mask into a 2D-array
            #we need to create a masker-object using the beforehand created mask of the pSTS as mask_img 
            masker = NiftiMasker(mask_img=pSTS_small, standardize=True)
        if mask == 'whb':
            #we use a masker-object to transform the data within the mask into a 2D-array
            #we need to create a masker-object using the beforehand created mask of the pSTS as mask_img 
            masker = NiftiMasker(mask_img=whb_mask, standardize=True)
        if mask == 'ff_pSTS':
            masker = NiftiMasker(mask_img=fusiform_pSTS_mask, standardize=True)
        if mask == 'lingual':
            masker = NiftiMasker(mask_img=lingual, standardize=True)
            
        if mask == 'whb_fmriprep':
            masker = NiftiMasker(mask_img=mask_filename,standardize=True)
        #we then use the fit.transform method of this masker-object to mask the data and then apply the tranformation
        #into a 2D-array afterwards
        #if that confuses you check out the nilearn documentation on nifti.maskers !
        fmri_masked = masker.fit_transform(data)

        #setting up the behavioral file such that the support vector machine can learn the labels 
        behavioral = pd.read_csv(path+"derivatives/timing_data/sub-"+sid+"/sub-"+sid+"_task-video_events_scan.csv", delimiter=',')

        conditions = behavioral['AU20']

        #as the scanner runs a few minutes longer than the paradigm we need to remove the scans aquired
        #after the experiment to not get into trouble with dimensions of arrays: input data must have as many 
        #timepoints as there are events, otherwise the SVM will not work (needs a label for every )
        fmri_masked = fmri_masked[np.arange(time_shift,len(conditions)+time_shift), :]

        #now that we have our data transformed into an 2D-array we can apply dimension reduction with a PCA
        #we use a full singular value decomposition which has as many components as needed to explain over 95%
        #of the variance in the data
        if pca_position == 'before_cutting' and pca == True:
            pca = PCA(n_components= 0.95, svd_solver='full')
            pca.fit(fmri_masked_cut)
            fmri_masked_cut = pca.transform(fmri_masked_cut)


        #reduce data to scans in which videos have been presented (e.g. rows containing 0 or 1)
        #first we have to generate a mask 
        condition_mask = conditions.isin([0, 1])
        total_runs = behavioral['run_total']
        total_runs = total_runs[condition_mask]

        fmri_masked_cut = fmri_masked[condition_mask]
        #now that we have our data transformed into an 2D-array we can apply dimension reduction with a PCA
        #we use a full singular value decomposition which has as many components as needed to explain over 95%
        #of the variance in the data
        if pca_position == 'after_cutting' and pca == True:
            pca = PCA(n_components= 0.95, svd_solver='full')
            pca.fit(fmri_masked_cut)
            fmri_masked_cut = pca.transform(fmri_masked_cut)
            
        for i in range(1,145):
                if i == 1:
                    run_mask = total_runs.isin([i])
                    fmri_run = fmri_masked_cut[run_mask]
                    fmri_run_av = np.mean(fmri_run, axis=0)
                elif i ==2:
                    run_mask = total_runs.isin([i])
                    fmri_run = fmri_masked_cut[run_mask]
                    fmri_run_av = np.stack((fmri_run_av,np.mean(fmri_run, axis=0)))
                else:
                    run_mask = total_runs.isin([i])
                    fmri_run = fmri_masked_cut[run_mask]
                    fmri_run_av = np.append(fmri_run_av,[np.mean(fmri_run, axis=0)],axis=0)
        #now that we have our data transformed into an 2D-array we can apply dimension reduction with a PCA
        #we use a full singular value decomposition which has as many components as needed to explain over 95%
        #of the variance in the data
        if pca_position == 'after_averaging' and pca == True:
            pca = PCA(n_components= 0.95, svd_solver='full')
            pca.fit(fmri_run_av)
            fmri_run_av = pca.transform(fmri_run_av)
            
        if pca == False:
            if method == 'block_average':
                print(fmri_run_av.shape)
                return [fmri_run_av,masker]
            else :
                print(fmri_masked_cut.shape)
                return [fmri_masked_cut,masker]

       
        if method == 'block_average':
                
            print(fmri_run_av.shape)
            return [fmri_run_av,masker,pca]
        else:
            print(fmri_masked_cut.shape)
            return [fmri_masked_cut,masker,pca]
        
        
    if isinstance(sub,list) == True:
        for sid in sub:
            data_array = data_transformer(sub=sid,method=method, mask=mask,time_shift=time_shift,pca=False)
            beh = pd.read_csv(path+"derivatives/timing_data/sub-"+sid+"/sub-"+sid+"_task-video_events_BlockAverage.csv", delimiter=',')
            conditions = beh['AU12']
            conditions_mask = conditions.isin([0,1])
            conditions_cut = conditions[conditions_mask]

            if sid == '02' :

                fmri_masked_cut_all = data_array[0]
                print(fmri_masked_cut_all.shape)


            else:
                fmri_masked_cut_all=np.concatenate((fmri_masked_cut_all,data_array[0]),axis=0)
                print(fmri_masked_cut_all.shape)
        if pca == True:
            pca = PCA(n_components= 0.95, svd_solver='full')
            pca.fit(fmri_masked_cut_all)
            fmri_masked_cut_all = pca.transform(fmri_masked_cut_all)
            print(fmri_masked_cut_all.shape)
            return [fmri_masked_cut_all,pca]
        else:
            
            return fmri_masked_cut_all






## Define function for classification
This method requires a data-array and either a single subject_id or a list of subject_ids as mandatory inputs.
If a single subject_id is given this method will run a classification with a leave-one-run-out crossvalidation (several classifiers possible, default = svc) over all AU's to be investigated in this work **(AU1,AU2,AU12,AU20)** and print all the outputs of the different splits and a mean accuracy of the classifier.
We need to have the parameter method again as this function automatically creates the label files and they differ if we average over blocks (only 144 events compared to >1000 events for the single scan approach).

If this method is provided with a list of subject_ids it will run a group-level classification comparing between subjects using a leave-one-subject-out cross validation. This is done for every AU of interest.

To add together the label-files of all subjects in the list this function calls the function get_targets (see documentation below) which will output either a file with all the labels or a file with information about the subjects (necessary for CV-method).

In [26]:
def classification(sub,data_array,method='single_scan',classifier='svc',get_weights=False):
    
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import AdaBoostClassifier
    clf1 = RandomForestClassifier(max_depth=50, random_state=0, class_weight='balanced')
    clf2 = DecisionTreeClassifier(random_state=0, class_weight='balanced')
    clf3 = AdaBoostClassifier(n_estimators=100, random_state=0)
    svc = SVC(kernel='linear',class_weight='balanced')
    cv = LeaveOneGroupOut()
    cv2 = StratifiedKFold(n_splits=9)

    if classifier == 'svc':
            clf = svc
    if classifier == 'rfc':
            clf = clf1
    if classifier == 'dtc':
            clf = clf2
    if classifier == 'adaboost':
            clf = clf3
    
    if isinstance(sub,str):
        sid = sub

        #save the data given to the function as a variable (is this necessary?)  
        if method == 'block_average':
            beh = pd.read_csv(path+"derivatives/timing_data/sub-"+sid+"/sub-"+sid+"_task-video_events_BlockAverage.csv", delimiter=',')
        else:
            beh = pd.read_csv(path+"derivatives/timing_data/sub-"+sid+"/sub-"+sid+"_task-video_events_scan.csv", delimiter=',')
            conditions = beh['AU20']
            condition_mask = conditions.isin([0, 1])
            beh = beh[condition_mask]


        for AU in AUs:
            labels = beh[AU]
            #we need to extract information of runs for beeing able to perform leave-one-run-out CV
            cv_file = beh['run']
            #the function cross_val_score does all the magic: it uses the svc to train the labels of the conditions file
            #on the (masked) data the crossvalidation method is defined by cv (LOGO) the groups parameter is defined
            #by the runs_label as we want a leave-one-group-out CV
            #the last parameter is used to speed up things a little, enabling all processors
            cv_score_pca = cross_validate(clf,
                                        data_array,
                                        labels,
                                        cv=cv2,
                                        #groups=cv_file,
                                        return_estimator=True,
                                        n_jobs=-1
                                        )
            #we finally print the different scores of the different splittings, calculate their mean and also print this
            #need to make things a little nicer to look at here !
            print("Scores of the ndifferent splits of the Cross-Validation with PCA:")
            
            print(cv_score_pca['test_score'])
            
            print("model weights:")
            i=1
            for model in cv_score_pca['estimator']:
                if i == 1:
                    weights = model.coef_
                else:
                    weights = np.append(weights,model.coef_,axis=0)
                    
                i = i+1
                #weights = np.mean(weights, weights)
            #print(weights)
            #print(weights.shape)
            print("\n")
            weights_av = np.mean(weights,axis=0)
            #weights_av.tofile('weights_AU'+AU+'.csv',sep=',',format='%10.5f')
            #weights_av = weights_av.tolist()
            print("Averaged weights:")
            #print(weights_av)
            #print(weights_av.shape)
            #print(type(weights_av))
            if get_weights == True:
                weights_av.tofile('/media/lmn/86A406A0A406933B/Aaron_MA/data_bids/derivatives/results/sub-'+sid+'/weights_'+AU+'.csv',sep=',',format='%10.5f')
            
            #max_values = Nmaxelements(weights_av,40)
            #print(max_values)


            #print(cv_score_pca)
            max_values_index = np.argpartition(weights_av, -20)[-20:]
            print(max_values_index)

            accuracy_pca = cv_score_pca['test_score'].sum() / float(len(cv_score_pca['test_score']))
            print ("\nMean accuracy of "+ classifier+" with PCA for "+AU+ " is:")
            print('\x1b[0;31;43m' + str(accuracy_pca) + '\x1b[0m')
            print("\n")

            x=[]
            label_on = labels.isin([1])
            label_off = labels.isin([0])
            for i in range(data_array.shape[1]):
                x.append(i)

            #plt.plot(x, np.min(fmri_masked,axis=0), 'ro')
            #plt.plot(x, np.max(fmri_masked,axis=0), 'bo')
            plt.plot(x, np.mean(data_array[label_off],axis=0), 'ro')
            plt.plot(x, np.mean(data_array[label_on],axis=0), 'go')
            plt.show()
            
    if isinstance(sub,list):
        
        
        
        #between m,ethods
        
        for AU in AUs:
            
            labels = get_targets(sub=sub,AU=AU,targets='target',method=method)
            print(labels.shape)
            cv_file= get_targets(sub=sub,AU=AU,targets='sub_label',method=method)
            
            cv_score_pca = cross_val_score(clf,
                                data_array,
                                labels,
                                cv=cv2,
                                #groups=cv_file,
                                n_jobs=-1
                                )
            #we finally print the different scores of the different splittings, calculate their mean and also print this
            #need to make things a little nicer to look at here !
            print("Scores of the ndifferent splits of the Cross-Validation with PCA:")
            print(cv_score_pca)

            accuracy_pca = cv_score_pca.sum() / float(len(cv_score_pca))
            print ("\nMean accuracy of "+ classifier+" with PCA for "+AU+ " is:")
            print('\x1b[0;31;43m' + str(accuracy_pca) + '\x1b[0m')
            print("\n")

            x=[]
            label_on = labels.isin([1])
            label_off = labels.isin([0])
            for i in range(data_array.shape[1]):
                x.append(i)

            #plt.plot(x, np.min(fmri_masked,axis=0), 'ro')
            #plt.plot(x, np.max(fmri_masked,axis=0), 'bo')
            plt.plot(x, np.mean(data_array[label_off],axis=0), 'ro')
            plt.plot(x, np.mean(data_array[label_on],axis=0), 'go')
            plt.show()
            #idea: save 







## Define a function for the classification with permutation tests:
This function does the same as the above defined classification fucntion but also applies permutation testing for every classifier.

In [104]:
def classification_perm(sub,AU,data_array,method='single_scan',classifier='svc',get_weights=False):
    
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import AdaBoostClassifier
    clf1 = RandomForestClassifier(max_depth=50, random_state=0, class_weight='balanced')
    clf2 = DecisionTreeClassifier(random_state=0, class_weight='balanced')
    clf3 = AdaBoostClassifier(n_estimators=100, random_state=0)
    svc = SVC(kernel='linear',class_weight='balanced')
    cv = LeaveOneGroupOut()
    cv2 = StratifiedKFold(n_splits=9)

    if classifier == 'svc':
            clf = svc
    if classifier == 'rfc':
            clf = clf1
    if classifier == 'dtc':
            clf = clf2
    if classifier == 'adaboost':
            clf = clf3
    
    if isinstance(sub,str):
        sid = sub

        #save the data given to the function as a variable (is this necessary?)  
        if method == 'block_average':
            beh = pd.read_csv(path+"derivatives/timing_data/sub-"+sid+"/sub-"+sid+"_task-video_events_BlockAverage.csv", delimiter=',')
        else:
            beh = pd.read_csv(path+"derivatives/timing_data/sub-"+sid+"/sub-"+sid+"_task-video_events_scan.csv", delimiter=',')
            conditions = beh['AU20']
            condition_mask = conditions.isin([0, 1])
            beh = beh[condition_mask]
            
        labels = beh[AU]
        #we need to extract information of runs for beeing able to perform leave-one-run-out CV
        cv_file = beh['run']

        #the function cross_val_score does all the magic: it uses the svc to train the labels of the conditions file
        #on the (masked) data the crossvalidation method is defined by cv (LOGO) the groups parameter is defined
        #by the runs_label as we want a leave-one-group-out CV
        #the last parameter is used to speed up things a little, enabling all processors
        score,permutation_scores, pvalue = permutation_test_score(clf,
                                    data_array,
                                    labels,
                                    cv=cv,
                                    groups=cv_file,
                                    #return_estimator=True,
                                    scoring= 'accuracy' ,
                                    n_permutations= 1000,                              
                                    n_jobs=-1
                                    )
        
        #we finally print the different scores of the different splittings, calculate their mean and also print this            #need to make things a little nicer to look at here !
        print("Scores of the ndifferent splits of the Cross-Validation with PCA:")
        score_threshhold = Nmaxelements(permutation_scores.tolist(),50)
        mean = sum(score_threshhold)/len(score_threshhold)
        mean_all_perm=permutation_scores.sum()/len(permutation_scores)
        print(score)
        print(score_threshhold)
        print(mean)
        print(mean_all_perm)
        print(pvalue)
            
        # #############################################################################
        # View histogram of permutation scores
        plt.hist(permutation_scores, 20, label='Permutation scores',
                 edgecolor='black')
        ylim = plt.ylim()
        # BUG: vlines(..., linestyle='--') fails on older versions of matplotlib
        # plt.vlines(score, ylim[0], ylim[1], linestyle='--',
        #          color='g', linewidth=3, label='Classification Score'
        #          ' (pvalue %s)' % pvalue)
        # plt.vlines(1.0 / n_classes, ylim[0], ylim[1], linestyle='--',
        #          color='k', linewidth=3, label='Luck')
        plt.plot(2 * [score], ylim, '--g', linewidth=3,
                    label='Classification Score'
                 ' (pvalue %s)' % pvalue)
        plt.plot(2 * [1. / 2], ylim, '--k', linewidth=3, label='Luck')
        plt.ylim(ylim)
        plt.legend()
        plt.xlabel('Score')
        plt.show()
        
        return [score,mean_all_perm,pvalue,permutation_scores]




        
            
            
           
            
    if isinstance(sub,list):
        
        
        
        #between m,ethods
        
        
            
        labels = get_targets(sub=sub,AU=AU,targets='target',method=method)
        cv_file= get_targets(sub=sub,AU=AU,targets='sub_label',method=method)
            
        score,permutation_scores, pvalue = permutation_test_score(clf,
                                    data_array,
                                    labels,
                                    cv=cv2,
                                    #groups=cv_file,
                                    #return_estimator=True,
                                    scoring= 'accuracy' ,
                                    n_permutations= 1000,                              
                                    n_jobs=-1
                                    )
        
        mean_all_perm=permutation_scores.sum()/len(permutation_scores)
        
        
        # #############################################################################
        # View histogram of permutation scores
        plt.hist(permutation_scores, 20, label='Permutation scores',
                 edgecolor='black')
        ylim = plt.ylim()
        # BUG: vlines(..., linestyle='--') fails on older versions of matplotlib
        # plt.vlines(score, ylim[0], ylim[1], linestyle='--',
        #          color='g', linewidth=3, label='Classification Score'
        #          ' (pvalue %s)' % pvalue)
        # plt.vlines(1.0 / n_classes, ylim[0], ylim[1], linestyle='--',
        #          color='k', linewidth=3, label='Luck')
        plt.plot(2 * [score], ylim, '--g', linewidth=3,
                    label='Classification Score'
                 ' (pvalue %s)' % pvalue)
        plt.plot(2 * [1. / 2], ylim, '--k', linewidth=3, label='Luck')
        plt.ylim(ylim)
        plt.legend()
        plt.xlabel('Score')
        plt.show()
        
        return [score,mean_all_perm,pvalue,permutation_scores]
            






## Define a function for the backmapping of classifier weights
This function uses the information of the Nifti Masker and the pca, if applied, to map the weights of the classifier back into MNI space. SInce all the steps done befor in the analysis are linear this is achieved by inversly transforming the PCA (if aplied) and the Nifti Masker transformations

In [10]:
def inverse_transformer(sid, AUs,masker,path=path,pca='None'):
    if isinstance(AUs, str):
        AU = AUs
        #data =  nilearn.image.load_img(path+"derivatives/fmriprep/sub-"+sid+"/func/sub-"+sid+"_task-video_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz", dtype="auto")
        weights = pd.read_csv('/media/lmn/86A406A0A406933B/Aaron_MA/data_bids/derivatives/results/sub-'+sid+'/weights_'+AU+'.csv', delimiter=',', header= None)
        discriminant_voxels = weights.values
        if pca != 'None':
            discriminant_voxels = pca.inverse_transform(discriminant_voxels)
        MDV_volume = masker.inverse_transform(discriminant_voxels)
        return MDV_volume
    if isinstance(AUs,list):
        counter = 1
        for AU in AUs:

            if counter ==1 :
                weights_av = inverse_transformer(sid=sid,masker=masker,AUs =AU,path=path,pca=pca)
                counter = counter +1
            elif counter ==2:
                temporal = inverse_transformer(sid=sid,masker=masker, AUs=AU,path=path,pca=pca)
                weights_av = np.stack(weights_av,temporal)
            else :
                temporal = inverse_transformer(sid=sid,masker=masker, AUs=AU,path=path,pca=pca)
                weights_av = np.append(weights_av,[temporal],axis=0)
                
        weights_av = np.mean(weights_av,axis=0)
        return weights_av

    
    
    

## Define a function to get the targets for the between-subject analysis

In [9]:
def get_targets(sub,AU,targets,method='single_scan'):
    for sid in sub:
        if method == 'block_average':
            beh = pd.read_csv(path+"derivatives/timing_data/sub-"+sid+"/sub-"+sid+"_task-video_events_BlockAverage.csv", delimiter=',')
        else:
            beh = pd.read_csv(path+"derivatives/timing_data/sub-"+sid+"/sub-"+sid+"_task-video_events_scan.csv", delimiter=',')
            conditions = beh[AU]
            condition_mask = conditions.isin([0, 1])
            beh = beh[condition_mask]
        subject_label = beh['subject_id']
        conditions = beh[AU]
        
        if sid == '02' :

            #fmri_masked_cut_all = data_array
            subject_label_all = subject_label
            conditions_cut_all= conditions

        else:
            #fmri_masked_cut_all=np.concatenate((fmri_masked_cut_all,data_array),axis=0)
            subject_label_all=pd.concat([subject_label_all,subject_label], axis=0)
            conditions_cut_all=pd.concat((conditions_cut_all,conditions),axis=0)
    if targets == 'sub_label' : 
        print(type(subject_label_all))
        return subject_label_all
    if targets == 'target':
        print(type(conditions_cut_all))
        return conditions_cut_all
    


## Define a main fuction combining the functions into a single one
This function needs a path an a list of subject_ids which should be directories in the given path.
One can then choose to do a within-subject or a between-subject analysis.
For thenwithin-subject design the function loops over every subject_id in the given list performing a classificatio of the return data of data_transformer using the parameters set in the main function for the classification and data_transformer function.
For the between subject the main function passes the list over to the classification and data_transformer, such that the data_tranformer will return a data-arry which is the concatenation of each subject's data-array. Classification will use the target files for the bewteen_subject level (see documenation above).  
**Mandatory parameters of the function:** 


**path** : the full path of the directory to work with (needs full path) 

**sub** : a list of subject_ids you want to run the analysis on (needs a list of strings)

**Optional parameters of the function:**

**level** : you can choose between 'within_sub' or 'between_sub' 

**time_shift** : number of scans that you want to shift your fmri-data in order to account for the hemodynamic response (needs an integer > 0 , default = 0) 

**mask** : you can choose 'whb' (wholebrain) , 'pSTS' (default), 'ff_pSTS' (fusiform+ pSTS), 'lingual' or pass your own binary mask (tbd)  

**method** : 'single_scan' (default) for counting every scan as an indenpent event or 'block_average' to average the BOLD signal over blocks (e.g. get a mean signal for every block as one event)  

**pca** : 'before_cutting' , 'after_cutting', 'after_averaging'  

**classifier** : 'svc' (support vector machine), 'rfc' (random forest classifier), 'dtc' (decision tree classifier), 'adaboost'



In [2]:
def main(path, sub, level='within_sub',time_shift=0, mask='pSTS',method='single_scan', pca=True,pca_position='after_cutting',classifier='svc',permutations=True,get_MDV=True):
    
      
    if level == 'within_sub':
        timeshift = str(time_shift)
        for sid in sub:
            print('Subject: '+sid)
            data = data_transformer(sub=sid, method=method, mask=mask,pca=pca,pca_position=pca_position,time_shift=time_shift)
            if permutations == True:
                for AU in AUs:
                    print('Action Unit:  '+ AU)
                    results = classification_perm(sub=sid,AU=AU,data_array=data[0],method=method,get_weights=get_MDV,classifier=classifier)
                    for index,row in results_df.iterrows():
                        if row['sid'] == sid and row['AU'] == AU :
                            results_df.at[index, 'score'] = results[0]
                            results_df.at[index, 'pvalue'] = results[2]
                            results_df.at[index, 'permutation_score'] = results[1]
                    print(type(results[0]))
                    print(type(results[1]))
                    print(type(results[2]))
            else:
                for AU in AUs:
                    print('Action Unit:  '+ AU)
                    results = classification(sub=sid,AU=AU,data_array=data[0],method=method,get_weights=get_MDV,classifier=classifier)
                    for index,row in results_df.iterrows():
                        if row['sid'] == sid and row['AU'] == AU :
                            results_df.at[index, 'score'] = results[0]
                            results_df.at[index, 'pvalue'] = results[2]
                            results_df.at[index, 'permutation_score'] = results[1]
                    print(type(results[0]))
                    print(type(results[1]))
                    print(type(results[2]))
            
            if get_MDV == True:
                if pca == False:
                    for AU in AUs:
                        mdv = inverse_transformer(sid=sid,AUs = AU, masker = data[1])
                        nib.save(mdv,path+'/derivatives/results/sub-'+sid+'/weights_'+AU+'_mask_'+mask+'_method_'+method+'_clf_'+classifier+'_timeshift-'+timeshift+'.nii.gz')
                        #mdv_av = inverse_transformer(sid=sid, AUs=AUs,nasker=data[1])
                        #nib.save(mdv_av,path+'/derivatives/results/sub-'+sid+'/weights_av_mask_'+mask+'_method_'+method+'_clf_'+classifier+'.nii.gz')
                else:
                    for AU in AUs:
                        mdv = inverse_transformer(sid=sid,AUs=AU,masker=data[1],pca=data[2])
                        nib.save(mdv,path+'/derivatives/results/sub-'+sid+'/weights_'+AU+'_mask_'+mask+'_method_'+method+'_pca_'+pca_position+'_clf_'+classifier+'_timeshift-'+timeshift+'.nii.gz')
                        #mdv = inverse_transformer(sid=sid,AUs=AUs,masker=data[1],pca=data[2])
                        #nib.save(mdv,path+'/derivatives/results/sub-'+sid+'/weights_av_mask_'+mask+'_method_'+method+'_pca_'+pca_position+'_clf_'+classifier+'.nii.gz')
                
    results_df.to_csv(path+'/derivatives/results/results_all_mask-'+mask+'_method-'+method+'_pca-'+pca_position+'_clf-'+classifier+'_timeshift_'+timeshift+'.csv')                             
                
            
    if level == 'between_sub':
        data_all=data_transformer(sub=sub,method=method,mask=mask,pca=pca,time_shift=time_shift)
        classification(sub=sub,data_array=data_all,classifier=classifier,method=method)
        
        

In [18]:
def Nmaxelements(list1, N): 
    final_list = [] 
  
    for i in range(0, N):  
        max1 = 0
          
        for j in range(len(list1)):      
            if list1[j] > max1: 
                max1 = list1[j]; 
                  
        list1.remove(max1); 
        final_list.append(max1) 
          
    return(final_list) 